In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import regexp_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import gensim.corpora as corpora
import gensim
from datetime import datetime, timedelta
import time
import pickle
import pprint

import warnings
warnings.filterwarnings('ignore')

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fergu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\fergu\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [4]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\fergu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
wnl = WordNetLemmatizer()

def func_lemmatize(words):
    lemmatized = []
    for word, tag in pos_tag(words):
        wntag = tag[0].lower()
        wntag = wntag if wntag in ['a','r','n','v'] else None
        
        lemma = wnl.lemmatize(word,wntag) if wntag else word
        lemmatized.append(lemma)
    return lemmatized

stop = set(stopwords.words('english'))

pattern = r'(\w+)'

In [46]:
def func_lemmatize(words):
    allow = ['a','r','n','v']
    
#    list_1 = [(word,tag[0].lower() if tag[0].lower() in allow else None) for word,tag in pos_tag(words)]
    list_2 = [wnl.lemmatize(word,wntag) if wntag else word for word, wntag in [(word,tag[0].lower() if tag[0].lower() in allow else None) for word,tag in pos_tag(words)]]
    
    return list_2

In [15]:
def func_lemmatize(words):
    allow = ['a','r','n','v']
    
    list_1 = [(word,tag[0].lower() if tag[0].lower() in allow else None) for word,tag in pos_tag(words)]
    list_2 = [wnl.lemmatize(word,wntag) if wntag else word for word, wntag in list_1]
    
    return list_2

In [ ]:
def func_lemmatize(words):
    allow = ['a','r','n','v']
    
    lemmatized = [wntag if wntag[0].lower() in allow for word,tag in pog_tag(words)]
    lemma = [wnl.lemmatize(item) for item in list_1]

In [6]:
# 12:05 (5 mins)
#df = pd.read_csv('all-the-news-2-1.csv')
df= pd.read_csv('../all-the-news-2-1.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,date,year,month,day,author,title,article,url,section,publication
0,0,0,2016-12-09 18:31:00,2016,12.0,9,Lee Drutman,We should take concerns about the health of li...,"This post is part of Polyarchy, an independent...",https://www.vox.com/polyarchy/2016/12/9/138983...,NaN,Vox
1,1,1,2016-10-07 21:26:46,2016,10.0,7,Scott Davis,Colts GM Ryan Grigson says Andrew Luck's contr...,The Indianapolis Colts made Andrew Luck the h...,https://www.businessinsider.com/colts-gm-ryan-...,NaN,Business Insider
2,2,2,2018-01-26 00:00:00,2018,1.0,26,NaN,Trump denies report he ordered Mueller fired,"DAVOS, Switzerland (Reuters) - U.S. President ...",https://www.reuters.com/article/us-davos-meeti...,Davos,Reuters
3,3,3,2019-06-27 00:00:00,2019,6.0,27,NaN,France's Sarkozy reveals his 'Passions' but in...,PARIS (Reuters) - Former French president Nico...,https://www.reuters.com/article/france-politic...,World News,Reuters
4,4,4,2016-01-27 00:00:00,2016,1.0,27,NaN,Paris Hilton: Woman In Black For Uncle Monty's...,Paris Hilton arrived at LAX Wednesday dressed ...,https://www.tmz.com/2016/01/27/paris-hilton-mo...,NaN,TMZ


In [7]:
df.shape

(2688879, 12)

In [8]:
df = df.dropna(subset = ['article'])

In [9]:
df.reset_index(inplace=True,drop=True)

In [10]:
df.shape

(2584165, 12)

In [11]:
## df1 = df[:250000]
## df2 = df[250000:500000]
## df3 = df[500000:750000]
## df4 = df[750000:1000000]
## df5 = df[1000000:1250000]
## df6 = df[1250000:1500000]
## df7 = df[1500000:1750000]
## df8 = df[1750000:2000000]
## df9 = df[2000000:2250000]
## df10 = df[2250000:]

In [12]:
num = 250000
np.random.seed(3)
inds_to_use = np.random.choice(df.index,num,replace=False)
df2 = df.loc[inds_to_use,:]
#df2 = df.loc[:num]
df2.reset_index(drop=True,inplace=True)

In [13]:
len(df2)

250000

In [75]:
def first_nth(string, substring, n):
    if (n == 1):
        print()
        return string.find(substring)     
    else:
        return string.find(substring, first_nth(string, substring, n - 1) + 1)

In [74]:
a = 'hello how are you today my friend'
first_nth(a,' ',3)

13

In [118]:
df2.shape

(250001, 12)

In [27]:
## def clean(df):
##     start = time.time()
##     df['article_words'] = [regexp_tokenize(article,pattern) for article in df['article']]
##     df['article_words'] = [[word for word in article if word.isalpha()] for article in df['article_words']]
##     df['article_words'] = [func_lemmatize(article) for article in df['article_words']]
##     df['article_words'] = [[word.lower() for word in article if word.lower() not in stop] for article in df['article_words']]
##     interval = round((time.time() - start)/60,2)
##     print(f'That  took {interval} mins.')
##     return df

In [19]:
def clean(df):
    start = time.time()
    df['article_words'] = [regexp_tokenize(article,pattern) for article in df['article']]
    df['article_para'] = [article[:120] for article in df['article_words']]
    df['article_title'] = [regexp_tokenize(title,pattern) for title in df['title']]
    df['summary'] = [df['article_title'][i] + df['article_para'][i] for i in range(len(df))]
    
    df['summary_words'] = [[word for word in article if word.isalpha()] for article in df['summary']]
    df['summary_words'] = [func_lemmatize(article) for article in df['summary_words']]
    df['summary_words'] = [[word.lower() for word in article if word.lower() not in stop] for article in df['summary_words']]
    interval = round((time.time() - start)/60,2)
    print(f'That  took {interval} mins.')
    return df

In [14]:
# Original version
def clean(df):
    start = time.time()
    df['article_words'] = [regexp_tokenize(article,pattern) for article in df['article']]
    print('tokenizing done.')  
    df['article_words'] = [[word for word in article if word.isalpha()] for article in df['article_words']]
    print('isalpha done.')
    df['article_words'] = [func_lemmatize(article) for article in df['article_words']]
    print('lemmatizing done.')
    df['article_words'] = [[word.lower() for word in article if word.lower() not in stop] for article in df['article_words']]
    interval = round((time.time() - start)/60,2)
    print(f'That  took {interval} mins.')
    return df

In [32]:
## def clean(df,n=120):
##     start = time.time()
##     df['trunc_article'] = [article[:first_nth(str(article),' ',n)] for article in df['article']]
##     
##     df['article_words'] = [regexp_tokenize(article,pattern) for article in df['trunc_article']]
## #    df['article_para'] = [article[:120] for article in df['article_words']]
##     df['article_title'] = [regexp_tokenize(title,pattern) for title in df['title']]
##     df['summary'] = [df['article_title'][i] + df['article_para'][i] for i in range(len(df))]
##     
##     df['summary_words'] = [[word for word in article if word.isalpha()] for article in df['summary']]
##     df['summary_words'] = [func_lemmatize(article) for article in df['summary_words']]
##     df['summary_words'] = [[word.lower() for word in article if word.lower() not in stop] for article in df['summary_words']]
##     interval = round((time.time() - start)/60,2)
##     print(f'That  took {interval} mins.')
##     return df

In [49]:
# One list comp
# 11:26
df2 = clean(df2)

tokenizing done.
isalpha done.
lemmatizing done.
That  took 10.83 mins.


In [ ]:
# Two list comps
# 22:20
df2 = clean(df2)

tokenizing done.
isalpha done.


In [15]:
# Original Function
# 09:24
df2 = clean(df2)

tokenizing done.
isalpha done.
lemmatizing done.
That  took 133.46 mins.


In [16]:
df2.head()

,Unnamed: 0,Unnamed: 0.1,date,year,month,day,author,title,article,url,section,publication,article_words
0,539903,539903,2017-06-01 00:00:00,2017,6.0,1,James Vincent,Misfit’s new tool lets you customize your fitn...,Misfit has launched a new tool that lets users...,https://www.theverge.com/circuitbreaker/2017/6...,Tech,The Verge,"[misfit, launch, new, tool, let, user, persona..."
1,255873,255873,2019-07-11 00:00:00,2019,7.0,11,Marcelo Teixeira,"Coffee farmers to form legal entity, launch pl...","CAMPINAS, Brazil, July 11 (Reuters) - Particip...",https://www.reuters.com/article/coffee-forum-o...,Non-Cyclical Consumer Goods,Reuters,"[campinas, brazil, july, reuters, participants..."
2,2403288,2415652,2019-11-06 00:00:00,2019,11.0,6,NaN,China supports more aggressive measures to tam...,"BEIJING, Nov 6 (Reuters) - China’s Vice Premie...",https://www.reuters.com/article/hongkong-prote...,Cyclical Consumer Goods,Reuters,"[beijing, nov, reuters, china, vice, premier, ..."
3,1328201,1328201,2017-01-11,2017,1.0,11,NaN,BRIEF-Selz Capital LLC reports 7.1 pct stake i...,Jan 11 (Reuters) - WPX Energy Inc * Selz Capit...,http://www.reuters.com/article/idUSFWN1F10JT,Funds News,Reuters,"[jan, reuters, wpx, energy, inc, selz, capital..."
4,2069896,2069896,2017-11-29 22:00:01,2017,11.0,29,Deb Amlen,Sinking It,Wordplay THURSDAY PUZZLE — Today’s puzzle by T...,https://www.nytimes.com/2017/11/29/crosswords/...,crosswords,The New York Times,"[wordplay, thursday, puzzle, today, puzzle, tr..."


In [17]:
df2.head()

,Unnamed: 0,Unnamed: 0.1,date,year,month,day,author,title,article,url,section,publication,article_words
0,539903,539903,2017-06-01 00:00:00,2017,6.0,1,James Vincent,Misfit’s new tool lets you customize your fitn...,Misfit has launched a new tool that lets users...,https://www.theverge.com/circuitbreaker/2017/6...,Tech,The Verge,"[misfit, launch, new, tool, let, user, persona..."
1,255873,255873,2019-07-11 00:00:00,2019,7.0,11,Marcelo Teixeira,"Coffee farmers to form legal entity, launch pl...","CAMPINAS, Brazil, July 11 (Reuters) - Particip...",https://www.reuters.com/article/coffee-forum-o...,Non-Cyclical Consumer Goods,Reuters,"[campinas, brazil, july, reuters, participants..."
2,2403288,2415652,2019-11-06 00:00:00,2019,11.0,6,NaN,China supports more aggressive measures to tam...,"BEIJING, Nov 6 (Reuters) - China’s Vice Premie...",https://www.reuters.com/article/hongkong-prote...,Cyclical Consumer Goods,Reuters,"[beijing, nov, reuters, china, vice, premier, ..."
3,1328201,1328201,2017-01-11,2017,1.0,11,NaN,BRIEF-Selz Capital LLC reports 7.1 pct stake i...,Jan 11 (Reuters) - WPX Energy Inc * Selz Capit...,http://www.reuters.com/article/idUSFWN1F10JT,Funds News,Reuters,"[jan, reuters, wpx, energy, inc, selz, capital..."
4,2069896,2069896,2017-11-29 22:00:01,2017,11.0,29,Deb Amlen,Sinking It,Wordplay THURSDAY PUZZLE — Today’s puzzle by T...,https://www.nytimes.com/2017/11/29/crosswords/...,crosswords,The New York Times,"[wordplay, thursday, puzzle, today, puzzle, tr..."


In [18]:
df2.to_csv('250k_full.csv')

In [19]:
dft = df2.copy()

## DELETE

In [20]:
def lda_compute(df,num_topics,passes=20):
    start = time.time()
    id2word = corpora.Dictionary(df['article_words'])
    
    articles = df['article_words']
    
    
    corpus = [id2word.doc2bow(article) for article in articles]
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,id2word=id2word,num_topics=num_topics,
                                           passes=passes,random_state=4)
    
#    print(lda_model.print_topics())
    print('-' * 10)
    doc_lda = lda_model[corpus]
    
    lda_lists = [lda_model[article] for article in corpus]
    print(f'lda_lists[-1]: {lda_lists[-1]}')
    lda_coeffs = [[pair[1] for pair in article] for article in lda_lists]
    print(f'lda_coeffs[-1]: {lda_coeffs[-1]}')
    lda_inds = [[pair[0] for pair in article] for article in lda_lists]
    print(f'lda_inds[-1]: {lda_inds[-1]}')
    
    rogues = [i for i in range(len(lda_inds)) if len(lda_inds[i])==0]
    
    lda_lists = [lda_lists[i] for i in range(len(lda_lists)) if i not in rogues]
    lda_coeffs = [lda_coeffs[i] for i in range(len(lda_coeffs)) if i not in rogues]
    lda_inds = [lda_inds[i] for i in range(len(lda_inds)) if i not in rogues]
    
    df.drop(rogues,inplace=True,axis=0)
    
    
#    df_rogue = df.loc[rogues,:]
#    return df_rogue 
    
###    lda_argmax = [np.argmax(tup) if len(tup) < 0 else None for tup in lda_coeffs]
#    lda_args = [np.arsgort(tup)[::-1] for tup in lda_coeffs]
    
    lda_argmax = [np.argmax(tup) for tup in lda_coeffs]
    
    
    print(f'lda_argmax[-1]: {lda_argmax[-1]}')
#    lda_inds = [pair[0] for pair in lda_tuples]
#    print(lda_inds[-1])
#    print('-' * 10)
#    print(lda_argmax[-1])
#    print('-' * 10)
###    lda_topics = [lda_inds[i][lda_argmax[i]] if lda_argmax[i] else None for i in range(len(lda_lists))]
    lda_topics = [lda_inds[i][lda_argmax[i]] for i in range(len(lda_lists))]
    
    sec_threshold = 0.5
    
#    pdb.set_trace()
    
#    secondary_topics = [[coeff for coeff in lda_coeffs[i] if 1 > (coeff / lda_coeffs[i][lda_argmax[i]]) >= sec_threshold] for i in range(len(lda_lists))]
        
    
#    secondary_topics = [[coeff for coeff in lda_coeffs[i] if 1 > (coeff / lda_coeffs[i][lda_argmax[i]]) >= sec_threshold] for i in range(len(lda_lists))]
    
    
#    secondary topics = [[lda_inds[j] for j in range(len(lda_inds[i])) if lda_coeffs[j] / lda_coeffs[i][lda_argmax[i]] >= sec_threshold] for i in range(len(lda_lists))]      
    
    secondary_topics = [[pair[0] for pair in lda_lists[i] if 1 > pair[1] / lda_coeffs[i][lda_argmax[i]] >= sec_threshold] for i in range(len(lda_lists))]
    
    df['lda_topic'] = lda_topics
    
    df['other_topics'] = secondary_topics   
    
    interval = round((time.time() - start)/60,2)
    print(f'That  took {interval} mins.')
    return df, lda_model, corpus#, doc_lda

In [21]:
# 11:50
dft, lda_model, corpus = lda_compute(dft,185)

----------
lda_lists[-1]: [(4, 0.15963906), (50, 0.012434837), (68, 0.01520978), (75, 0.013484603), (102, 0.24442917), (103, 0.25668555), (107, 0.011359962), (125, 0.034166105), (134, 0.014621516), (142, 0.045045972), (158, 0.090134), (163, 0.047828484), (166, 0.043360956)]
lda_coeffs[-1]: [0.15963906, 0.012434837, 0.01520978, 0.013484603, 0.24442917, 0.25668555, 0.011359962, 0.034166105, 0.014621516, 0.045045972, 0.090134, 0.047828484, 0.043360956]
lda_inds[-1]: [4, 50, 68, 75, 102, 103, 107, 125, 134, 142, 158, 163, 166]
lda_argmax[-1]: 5
That  took 305.7 mins.


In [22]:
dft.to_csv('250k_full_lda_185.csv')
lda_model.save('250k_ldamodel_185')

In [26]:
with open("250k_full_corpus.txt", "wb") as fp:
    pickle.dump(corpus, fp)

In [25]:
corpus.to_pickle('250k_full_corpus.pkl')

AttributeError: 'list' object has no attribute 'to_pickle'

## /DELETE

In [105]:
for i in range(200):
    print(i,':',len(df2['summary_words'][i]))

0 : 74
1 : 73
2 : 83
3 : 73
4 : 78
5 : 67
6 : 74
7 : 80
8 : 79
9 : 72
10 : 71
11 : 36
12 : 83
13 : 84
14 : 75
15 : 84
16 : 78
17 : 66
18 : 85
19 : 78
20 : 17
21 : 78
22 : 80
23 : 86
24 : 76
25 : 79
26 : 46
27 : 54
28 : 69
29 : 81
30 : 62
31 : 74
32 : 74
33 : 62
34 : 32
35 : 83
36 : 74
37 : 39
38 : 70
39 : 65
40 : 61
41 : 73
42 : 91
43 : 70
44 : 79
45 : 78
46 : 73
47 : 76
48 : 86
49 : 82
50 : 65
51 : 78
52 : 80
53 : 78
54 : 84
55 : 75
56 : 82
57 : 69
58 : 80
59 : 78
60 : 74
61 : 72
62 : 81
63 : 72
64 : 69
65 : 87
66 : 79
67 : 73
68 : 73
69 : 74
70 : 75
71 : 75
72 : 88
73 : 78
74 : 78
75 : 76
76 : 84
77 : 84
78 : 60
79 : 73
80 : 78
81 : 64
82 : 39
83 : 70
84 : 67
85 : 82
86 : 70
87 : 76
88 : 40
89 : 78
90 : 73
91 : 88
92 : 89
93 : 99
94 : 76
95 : 74
96 : 76
97 : 63
98 : 75
99 : 77
100 : 95
101 : 76
102 : 82
103 : 77
104 : 73
105 : 71
106 : 68
107 : 75
108 : 75
109 : 57
110 : 47
111 : 56
112 : 88
113 : 87
114 : 80
115 : 85
116 : 78
117 : 67
118 : 82
119 : 82
120 : 82
121 : 67
122 : 90
123

In [106]:
for i in range(200):
    print(i,':',df2['summary_words'][i])

0 : ['rpt', 'column', 'china', 'metal', 'trade', 'fragment', 'multiple', 'narrative', 'andy', 'home', 'repeats', 'aug', 'column', 'without', 'change', 'opinion', 'express', 'author', 'columnist', 'reuters', 'cobalt', 'import', 'tmsnrt', 'rs', 'zinc', 'import', 'tmsnrt', 'rs', 'nickel', 'ore', 'import', 'tmsnrt', 'r', 'andy', 'home', 'london', 'aug', 'reuters', 'china', 'metal', 'trade', 'figure', 'spotlight', 'state', 'world', 'manufacture', 'engine', 'price', 'driver', 'monthly', 'update', 'country', 'seemingly', 'insatiable', 'appetite', 'import', 'metal', 'hardwired', 'bull', 'market', 'narrative', 'start', 'decade', 'drum', 'beat', 'copper', 'march', 'bull', 'peak', 'tonne', 'early', 'rest', 'metal', 'march']
1 : ['new', 'music', 'edna', 'vazquez', 'play', 'song', 'psicodélica', 'pickathon', 'pumphouse', 'month', 'bring', 'episode', 'pickathon', 'pumphouse', 'series', 'film', 'tiny', 'pumphouse', 'studio', 'ground', 'pickathon', 'music', 'festival', 'happy', 'valley', 'oregon', 'li

In [88]:
df2['article'][83]

"There are two types of people in this world: Those who live for the Halloween season — and everybody else. But even if the spooky festivities aren't your vibe, you still have a lot to look forward to this month. Most importantly, fall products. The biggest beauty retailers are rolling out new masks, lip colors, and hair offerings left and right — especially Sephora. From fall eyeshadow palettes to winter-proof body serums, there's something for everyone, ahead. At Refinery29, we’re here to help you navigate this overwhelming world of stuff. All of our market picks are independently selected and curated by the editorial team. If you buy something we link to on our site, Refinery29 may earn commission."

In [84]:
df2['article_words'][5]

['NEW',
 'YORK',
 'June',
 '19',
 'Reuters',
 'U',
 'S',
 'Treasury',
 'yields',
 'fell',
 'on',
 'Tuesday',
 'with',
 '10',
 'year',
 'yield',
 'hitting',
 'its',
 'lowest',
 'in',
 'over',
 'two',
 'weeks',
 'following',
 'U',
 'S',
 'President',
 'Donald',
 'Trump',
 's',
 'threat',
 'to',
 'slap',
 'tariffs',
 'on']

In [78]:
df2

,Unnamed: 0,Unnamed: 0.1,date,year,month,day,author,title,article,url,section,publication,article_words,article_para,article_title,summary,summary_words,trunc_article
0,1527764,1527764,2017-08-28,2017,8.0,28,NaN,RPT-COLUMN-China's metals trade fragments into...,(Repeats Aug. 25 column without change. The op...,https://www.reuters.com/article/china-metals-a...,Intel,Reuters,"[Repeats, Aug, 25, column, without, change, Th...","[Repeats, Aug, 25, column, without, change, Th...","[RPT, COLUMN, China, s, metals, trade, fragmen...","[RPT, COLUMN, China, s, metals, trade, fragmen...","[rpt, column, china, metal, trade, fragment, m...",(Repeats Aug. 25 column without change. The op...
1,168893,168893,2016-04-08 15:50:02,2016,4.0,8,David Roberts,New music for you: Edna Vazquez plays her song...,"Once a month (or so), I'm bringing you an epis...",https://www.vox.com/culture/2016/4/8/11391828/...,NaN,Vox,"[Once, a, month, or, so, I, m, bringing, you, ...","[Once, a, month, or, so, I, m, bringing, you, ...","[New, music, for, you, Edna, Vazquez, plays, h...","[New, music, for, you, Edna, Vazquez, plays, h...","[new, music, edna, vazquez, play, song, psicod...","Once a month (or so), I'm bringing you an epis..."
2,2001409,2001409,2016-09-09 00:35:26,2016,9.0,9,The Associated Press,One Wild Pitch Leads to Two Runs as Indians Be...,Sports Briefing The Indians scored two runs on...,http://www.nytimes.com/2016/09/09/sports/baseb...,sports,The New York Times,"[Sports, Briefing, The, Indians, scored, two, ...","[Sports, Briefing, The, Indians, scored, two, ...","[One, Wild, Pitch, Leads, to, Two, Runs, as, I...","[One, Wild, Pitch, Leads, to, Two, Runs, as, I...","[one, wild, pitch, leads, two, runs, indians, ...",Sports Briefing The Indians scored two runs on...
3,358141,358141,2018-03-01 09:09:08,2018,3.0,1,Ron Miller,Dropbox to add native G Suite integration in n...,It’s been an eventful week for Dropbox coming ...,https://techcrunch.com/2018/03/01/dropbox-to-a...,NaN,TechCrunch,"[It, s, been, an, eventful, week, for, Dropbox...","[It, s, been, an, eventful, week, for, Dropbox...","[Dropbox, to, add, native, G, Suite, integrati...","[Dropbox, to, add, native, G, Suite, integrati...","[dropbox, add, native, g, suite, integration, ...",It’s been an eventful week for Dropbox coming ...
4,307943,307943,2017-12-14 00:00:00,2017,12.0,14,Christopher Matthews,Walmart will start giving salary advances to 1...,Walmart announced Wednesday that it will start...,https://www.axios.com/walm-1513267182-9615b668...,NaN,Axios,"[Walmart, announced, Wednesday, that, it, will...","[Walmart, announced, Wednesday, that, it, will...","[Walmart, will, start, giving, salary, advance...","[Walmart, will, start, giving, salary, advance...","[walmart, start, give, salary, advance, millio...",Walmart announced Wednesday that it will start...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2743574,2744334,2020-03-16 00:00:00,2020,3.0,16,NaN,"EasyJet says could ground most of fleet, calls...","LONDON, March 16 (Reuters) - EasyJet, Europe’s...",https://www.reuters.com/article/health-coronav...,Company News,Reuters,"[LONDON, March, 16, Reuters, EasyJet, Europe, ...","[LONDON, March, 16, Reuters, EasyJet, Europe, ...","[EasyJet, says, could, ground, most, of, fleet...","[EasyJet, says, could, ground, most, of, fleet...","[easyjet, say, could, grind, fleet, call, govt...","LONDON, March 16 (Reuters) - EasyJet, Europe’s..."
4996,253010,253010,2018-08-20 00:00:00,2018,8.0,20,Paul Wallace,Commentary: Don’t cheer Greece’s bailout exit ...,For once Europe has an exit to celebrate rathe...,https://www.reuters.com/article/us-wallace-gre...,Davos,Reuters,"[For, once, Europe, has, an, exit, to, celebra...","[For, once, Europe, has, an, exit, to, celebra...","[Commentary, Don, t, cheer, Greece, s, bailout...","[Commentary, Don, t, cheer, Greece, s, bailout...","[commentary, cheer, greece, bailout, exit, yet...",For once Europe has an exit to celebrate rat

In [28]:
# Old way
df2 = clean(df2)

That  took 3.73 mins.


In [31]:
# Started at 13:59
# Estd finish time: 15:52
df2 = clean(df2)

That  took 1.25 mins.


In [52]:
def remove_head_parts(s, delim, n):
    return s.split(delim, n)#[0]

In [55]:
x = remove_head_parts('hello how are you today',' ',1)
x

['hello', 'how are you today']

In [46]:
df2.head(1)

,Unnamed: 0,Unnamed: 0.1,date,year,month,day,author,title,article,url,section,publication,article_words,article_para,article_title,summary,summary_words
0,1527764,1527764,2017-08-28,2017,8.0,28,NaN,RPT-COLUMN-China's metals trade fragments into...,(Repeats Aug. 25 column without change. The op...,https://www.reuters.com/article/china-metals-a...,Intel,Reuters,"[Repeats, Aug, 25, column, without, change, Th...","[Repeats, Aug, 25, column, without, change, Th...","[RPT, COLUMN, China, s, metals, trade, fragmen...","[RPT, COLUMN, China, s, metals, trade, fragmen...","[rpt, column, china, metal, trade, fragment, m..."


In [ ]:
df2.to_pickle('df1_250_500k.pkl')
df2.to_csv('df1_250_500k.csv')

In [23]:
df2.to_pickle('p25million.pkl')
df2.to_csv('p25million.csv')

In [18]:
df2.head(1)

,Unnamed: 0,Unnamed: 0.1,date,year,month,day,author,title,article,url,section,publication,article_words
0,1527764,1527764,2017-08-28,2017,8.0,28,NaN,RPT-COLUMN-China's metals trade fragments into...,(Repeats Aug. 25 column without change. The op...,https://www.reuters.com/article/china-metals-a...,Intel,Reuters,"[repeats, aug, column, without, change, opinio..."


In [26]:
def lda_compute(df,num_topics):
    start = time.time()
    id2word = corpora.Dictionary(df['article_words'])
    
    texts = df['article_words']
    
    
    corpus = [id2word.doc2bow(text) for text in texts]
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,id2word=id2word,num_topics=num_topics)
    
    print(lda_model.print_topics())
    print('-' * 10)
    doc_lda = lda_model[corpus]
    interval = round((time.time() - start)/60,2)
    print(f'That  took {interval} mins.')

In [28]:
lda_compute(df2,20)

[(0, '0.028*"say" + 0.009*"people" + 0.007*"health" + 0.004*"coronavirus" + 0.004*"year" + 0.004*"one" + 0.004*"case" + 0.004*"new" + 0.004*"go" + 0.004*"get"'), (1, '0.007*"use" + 0.007*"like" + 0.006*"one" + 0.006*"people" + 0.006*"say" + 0.005*"make" + 0.005*"new" + 0.005*"work" + 0.005*"time" + 0.005*"get"'), (2, '0.012*"say" + 0.008*"school" + 0.007*"student" + 0.005*"year" + 0.005*"state" + 0.005*"hill" + 0.004*"city" + 0.004*"new" + 0.004*"water" + 0.004*"home"'), (3, '0.011*"say" + 0.010*"market" + 0.010*"bank" + 0.009*"rate" + 0.008*"trade" + 0.008*"year" + 0.008*"u" + 0.006*"economy" + 0.005*"percent" + 0.005*"economic"'), (4, '0.008*"like" + 0.007*"say" + 0.007*"one" + 0.007*"get" + 0.006*"make" + 0.006*"go" + 0.005*"year" + 0.005*"time" + 0.005*"new" + 0.004*"show"'), (5, '0.013*"company" + 0.012*"say" + 0.006*"new" + 0.005*"use" + 0.005*"make" + 0.005*"one" + 0.005*"would" + 0.004*"business" + 0.004*"like" + 0.004*"year"'), (6, '0.012*"year" + 0.012*"percent" + 0.010*"say"

In [28]:
df.dropna(subset=['article'],inplace=True)
df.reset_index(drop=True,inplace=True)

In [29]:
df.shape

(2584165, 12)

In [10]:
## tab = pd.DataFrame({'article':['If the good guys are smart, this is the moment they swoop in and arrest everyone.',
##                                'Did you make it to the Allen & Co conference in Sun Valley, Idaho this past week?',
##                               'A pandemic era box office bonanza for Marvel’s Black Widow put a spring in Disney’s step today, though it hasn’t quite translated to gains for cinemas;',
##                               'This latest update, along with projections of positive economic growth, was met with notable optimism from the White House and record highs on Wall Street.',
##                               'In midst of this uneven recovery, and perhaps somewhat counterintuitively, others have noticed another pattern that may further elevate unemployment rates in the months to come:',
##                               'people are leaving their jobs.']})

In [ ]:
#11:30
df['article_words'] = [regexp_tokenize(article,pattern) for article in df['article']]
df['article_words'] = [[word for word in article if word.isalpha()] for article in df['article_words']]
df['article_words'] = [func_lemmatize(article) for article in df['article_words']]
df['article_words'] = [[word.lower() for word in article if word.lower() not in stop] for article in df['article_words']]

In [ ]:
df.to_pickle('df_cleaned.pkl')
df.to_csv('df_cleaned.csv')

In [ ]:
def create_w2v(col1,col2,max_epochs,vec_size,window,min_count,resume=False):
    now  = datetime.now()
    start_time_str = now.strftime('%H:%M:%S')
    print('started at',start_time_str)
        
    start_time = time.time()
    word_list = [[words for words in line] for line in col1]
    unpacked_word_list = [word for line in word_list for word in line]
    
    filename = 'model.sav'
    
    if resume == False:
        model = Word2Vec(col1,vector_size=vec_size,window=window,min_count=min_count,workers=-1)
        model.train(col1,total_examples=len(col1),epochs=max_epochs)
        print('word2vec model has been trained.')
        pickle.dump(model,open(filename,'wb'))
    else:
        file = open(filename,'rb')
        model = pickle.load(file)
        
    averaged_vectors = [] if resume == False else pd.read_csv('intermediate_results.csv',index_col=0)
    
    if resume == True:
        averaged_vectors = [list(averaged_vectors.loc[val,:]) for val in averaged_vectors.index]
    
    j = start_index= len(averaged_vectors)
    if j != 0:
        print(f'{round(j/len(col1),3) * 100}% complete.')
    start_checkpoint_time = time.time()
    
    progress_indicators = []
    
    
    for row in range(j,len(col1)):
        list_of_words = col1[row]
        list_of_vectors = [model.wv[word] for word in list_of_words if unpacked_word_list.count(word) >= min_count]
        avg_vector = np.mean(list_of_vectors,axis=0) if list_of_vectors else np.zeros(vec_size)
        averaged_vectors.append(avg_vector)
#        print(avg_vector)
        
        j += 1
#        print(f'{j} done')
        if j % 1000 == 0:
            intermediate_results = pd.DataFrame(averaged_vectors)
            intermediate_results.to_csv('intermediate_results.csv')
            
        percent_complete = round(j/len(col1) * 100)
        if (percent_complete % 1 == 0) & (percent_complete not in progress_indicators):
            cum_time_raw = (time.time() - start_time)/60
            cum_time = round((time.time() - start_time)/60,1)
            print('{}% complete. Time elapsed: {} min'.format(percent_complete,cum_time))
            progress_indicators.append(percent_complete)
            
            remaining_time = cum_time_raw * (len(col1) - j) / (j - start_index)
            
            est_finish_time = datetime.now() + timedelta(minutes=remaining_time)
            est_finish_time = est_finish_time.strftime('%D:%H:%M:%S')
            print(f'Estimated finish time: {est_finish_time}')
            print('-' * 15)
            
    df_of_vectors = pd.DataFrame(averaged_vectors)
        
    intermediate_table = pd.concat([col2,df_of_vectors],axis=1)
        
    rows_to_remove = [ind for ind in intermediate_table.index if intermediate_table.iloc[ind,2] == 0]
        
    removed_comments = pd.DataFrame(col2[rows_to_remove])
    final_table = intermediate_table.drop(rows_to_remove,axis=0)
        
    interval = round((time.time() - start_time)/60,1)
    print('-' * 25)
    print(f'Finished. That took {interval} min.')
    now = datetime.now()
    end_time_str = now.strftime('%H:%M:%S')
    print(f'Actual finish time: {end_time_str}')
        
    return final_table, removed_comments

In [ ]:
final_table, removed_comments = create_w2v(tab['article_words'],tab['tweet'],5,100,5,5)

In [142]:
final_table

,tweet,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,!!! RT @mayasolovely: As a woman you shouldn't...,0.010742,-0.004391,0.015802,-0.003414,0.003390,0.003548,-0.003043,-0.002796,-0.001108,...,-0.000829,0.012718,0.006761,-0.006497,-0.012051,0.007794,-0.009507,0.006585,0.005737,-0.001666
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,0.004841,0.004436,0.000914,-0.009751,0.005038,0.011150,0.011603,0.009655,-0.012505,...,0.004664,0.002429,0.014794,0.004940,0.001413,-0.004927,0.012201,0.005936,-0.018094,0.001071
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,0.008423,0.012927,0.011020,0.010654,0.000597,-0.006037,0.007699,0.020504,-0.008069,...,-0.005035,-0.017148,0.020670,0.006083,-0.009244,-0.002631,0.001638,-0.007432,-0.004601,-0.016549
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,0.018069,-0.003312,0.002020,-0.004119,0.017109,-0.017962,0.002340,0.022885,0.002644,...,0.005931,-0.010217,0.019906,-0.022391,0.001863,0.004285,0.007125,-0.005956,0.020242,-0.004850
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,-0.009344,0.005417,-0.008419,0.004743,0.009811,-0.017030,0.007335,0.005161,-0.005933,...,0.005729,-0.005238,-0.000171,-0.005781,-0.016183,-0.009473,0.009901,-0.018340,0.006494,-0.018538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,-0.003729,-0.017738,0.009045,-0.016504,-0.010244,-0.000089,0.005051,0.001840,0.020293,...,-0.001044,0.008122,0.019703,0.007250,-0.005047,0.009291,0.010963,-0.000282,-0.033845,-0.003717
24779,"you've gone and broke the wrong heart baby, an...",0.002734,-0.002392,0.011020,-0.018769,-0.008788,0.001414,0.003678,0.012647,0.002893,...,0.001392,-0.001854,-0.005640,-0.003610,0.009778,0.003094,-0.006933,0.004517,-0.003791,-0.002358
24780,young buck wanna eat!!.. dat nigguh like I ain...,0.006171,-0.002461,-0.018769,-0.001899,-0.009897,-0.001166,-0.000997,0.009243,-0.004765,...,0.022077,-0.010136,0.003253,0.004180,-0.009084,0.002988,0.000453,-0.005204,0.001637,0.012166
24781,youu got wild bitches tellin you lies,-0.032547,-0.003426,-0.006520,0.006167,0.011983,-0.023309,0.006597,0.013260,-0.012604,...,0.008047,-0.002982,-0.004045,0.015829,0.015913,-0.007844,0.015752,-0.005909,-0.016210,-0.011625


In [133]:
removed_comments

,tweet


In [123]:
df = pd.read_csv('labeled_data.csv')

In [124]:
df

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies
